# Introduction

Object detection is all about detecting objects in an image or video stream and keeping track of them.In this project we primarily focus on crafting an efficient system that detects and simultaneously counts the number of vehicles passing through a video stream. Vehicle counter systems have many real world applications such as in traffic monitoring and tool booth systems.

# Vehicle Counter

In [ ]:
# Importing required libraries
import numpy as np
import cv2

# Creating a video capture object to read the video file
cap = cv2.VideoCapture("C:\\Users\\sastr\\Downloads\\video.mp4")

# Defining minimum width and height of the object to be detected and line position to count vehicles
min_width = 80
min_height = 80
line_position = 550

# Creating a background subtraction object using MOG algorithm
algo = cv2.bgsegm.createBackgroundSubtractorMOG()

# Defining a function to calculate the center of the detected object
def counter(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx,cy

# Creating empty list to store centers of the detected objects
detect = []
# Defining offset to consider vehicles on the line
offset = 6
# Initializing counter variable
count = 0

# Starting an infinite loop to read frames from video
while True:
    # Reading a frame from the video
    ret,frame1 = cap.read()
    # Converting the frame to grayscale
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    # Applying Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(grey,(3,3),5)

    # Applying background subtraction to detect moving objects
    img_sub = algo.apply(blur)
    # Dilating the detected objects to fill gaps
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    # Applying morphological closing to remove small holes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilatada = cv2.morphologyEx(dilat,cv2.MORPH_CLOSE,kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)

    # Finding contours of the detected objects
    counter_shape,h = cv2.findContours(dilatada,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # Drawing a line to count vehicles
    cv2.line(frame1,(0,line_position),(1300,line_position),(0,0,255),2)

    # Looping through all the detected objects
    for (i,c) in enumerate(counter_shape):
        # Getting the coordinates and dimensions of the detected object
        x,y,w,h = cv2.boundingRect(c)
        # Validating the detected object based on minimum width and height
        validate_counter = (w>=min_width) and (w>= min_height)
        if not validate_counter:
            continue
        # Drawing a rectangle around the detected object
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,0,255))

        # Calculating the center of the detected object
        center = counter(x,y,w,h)
        # Adding the center of the detected object to the list
        detect.append(center)
        # Drawing a circle at the center of the detected object
        cv2.circle(frame1,center,4,(0,0,255),-1)

        # Looping through all the detected objects to check if they are on the line
        for (x,y) in detect:
            if(y>line_position-offset) and (y < line_position + offset):
                # Increasing the counter if the object is on the line
                count = count+1
                detect.remove((x, y))
                print("Vehicle Counter: " + str(count))
        cv2.line(frame1, (0, line_position), (1300, line_position), (0, 0, 255), 2)
    cv2.putText(frame1,"Vehicle Counter : "+str(count),(450,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),5)
    cv2.imshow("video original",frame1)
    if cv2.waitKey(1) == 13:
            break
cv2.destroyAllWindows()
cap.release()
